In [1]:
import pandas as pd
import numpy as np
import json
import torch

# Direct imports from the same directory
from tabddpm_adapter import TabDDPMAdapter
from tabddpm_benchmark import evaluate_tabddpm, print_evaluation_results
from tabddpm_utils import preprocess_data, get_tstr_results

In [2]:
# Load configuration
print("# Load configuration")
with open("config.json", "r") as f:
    config = json.load(f)

# Load configuration


In [3]:
# 1. Load and prepare the dataset
print("# 1. Load and prepare the dataset")
data_raw = pd.read_csv("letter-recognition.csv")
print(f"Columns in dataset: {data_raw.columns.tolist()}")
print(f"Dataset shape: {data_raw.shape}")
print(data_raw.head())

# 1. Load and prepare the dataset
Columns in dataset: ['letter', 'xbox ', 'ybox ', 'width ', 'height', 'onpix ', 'xbar ', 'ybar ', 'x2bar', 'y2bar ', 'xybar ', 'x2ybar', 'xy2bar', 'xedge ', 'xedgey', 'yedge ', 'yedgex']
Dataset shape: (20000, 17)
  letter  xbox   ybox   width   height  onpix   xbar   ybar   x2bar  y2bar   \
0      T      2      8       3       5       1      8     13      0       6   
1      I      5     12       3       7       2     10      5      5       4   
2      D      4     11       6       8       6     10      6      2       6   
3      N      7     11       6       6       3      5      9      4       6   
4      G      2      1       3       1       1      8      6      6       6   

   xybar   x2ybar  xy2bar  xedge   xedgey  yedge   yedgex  
0       6      10       8       0       8       0       8  
1      13       3       9       2       8       4      10  
2      10       3       7       3       7       3       9  
3       4       4      10       6     

In [4]:
# 2. Preprocess data and detect categorical columns
print("\n# 2. Preprocess data and detect categorical columns")
data, categorical_columns = preprocess_data(data_raw)
print(f"Detected categorical columns: {categorical_columns}")

2025-04-27 12:47:10,218 - INFO - Converted letter to category type (has 26 unique values)



# 2. Preprocess data and detect categorical columns
Detected categorical columns: ['letter']


In [5]:
# 3. Define the target column for this dataset
print("\n# 3. Define the target column for this dataset")
target_column = "letter"
print(f"Target column: {target_column}")


# 3. Define the target column for this dataset
Target column: letter


In [6]:
# 4. Split the data into features and target
print("\n# 4. Split the data into features and target")
X = data.drop(columns=[target_column])
y = data[target_column]
print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"Target distribution:\n{y.value_counts()}")


# 4. Split the data into features and target
Features shape: (20000, 16)
Target shape: (20000,)
Target distribution:
letter
U    813
D    805
P    803
T    796
M    792
A    789
X    787
Y    786
Q    783
N    783
F    775
G    773
E    768
B    766
V    764
L    761
R    758
I    755
O    753
W    752
S    748
J    747
K    739
C    736
H    734
Z    734
Name: count, dtype: int64


In [7]:
# 5. Initialize and train TabDDPM
print("\n# 5. Initialize and train TabDDPM")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tabddpm = TabDDPMAdapter(**config["tabddpm_params"], device=device)
print("Training TabDDPM model...")
tabddpm.fit(X, y)
print("Training completed")


# 5. Initialize and train TabDDPM
Training TabDDPM model...
Original data shape: (20000, 17), Target column: letter
Added StandardScaler for 16 numerical columns
Target 'letter' identified as categorical with 26 classes
Class mapping: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25}

Preprocessing Summary:
- Number of numerical features: 16
- Number of categorical features: 0
- Categorical columns: []
- Target column: letter
- Target type: Categorical
- Number of target classes: 26
X shape: (20000, 16), y shape: (20000,)
X_tensor shape: torch.Size([20000, 16]), y_tensor shape: torch.Size([20000])
X_tensor shape: torch.Size([20000, 16])
y_tensor shape: torch.Size([20000])


Epoch 299, Loss: 0.3208: 100%|██████████| 300/300 [07:26<00:00,  1.49s/it]

Training completed


In [8]:
# 6. Generate synthetic data
print("\n# 6. Generate synthetic data")
n_samples = 1000  
print(f"Generating {n_samples} synthetic samples...")
synthetic_data = tabddpm.generate(n_samples)
print(f"Generated {len(synthetic_data)} synthetic samples")
print("Synthetic data head:")
print(synthetic_data.head())


# 6. Generate synthetic data
Generating 1000 synthetic samples...
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Debug - out_dict type: <class 'torch.Tensor'>
Added target column 'letter' with 26 unique values
Final columns in synthetic data: ['xbox ', 'ybox ', 'width ', 'height', 'onpix ', 'xbar ', 'ybar ', 'x2bar', 'y2bar ', 'xybar ', 'x2ybar', 'xy2bar', 'xedge ', 'xedgey', 'yedge ', 'yedgex', 'letter']
Generated 1000 synthetic samples
Synthetic data head:
       xbox       ybox      width      height     onpix       xbar      ybar   \
0  10.273256  13.066230   8.174987   6.944760   4.849386   9.092016  3.710027   
1   2.869403   3.610033   3.348009   5.667185   0.851724   7.721162  4.400609   
2   5.739124   8.398993   8.156721   5.502726   6.389643  11.486719  6.060035   
3   1.527788   4.733185   3.262411   3.360594   2.849169   6.039143  6.717039   
4  16.500000  16.500000  16.500000  16.500000  16.500000  16.500000  3.769151   

       x2bar

In [9]:
# 7. Evaluate quality using TSTR and other metrics
print("\n# 7. Evaluate quality using TSTR and other metrics")
print("Running evaluation...")
evaluation_results = evaluate_tabddpm(data, synthetic_data, target_column=target_column)
print_evaluation_results(evaluation_results)


# 7. Evaluate quality using TSTR and other metrics
Running evaluation...


2025-04-27 13:16:55,090 - INFO - Encoded categorical target with mapping: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25}
c:\Users\kamal\OneDrive\Documents\Vilo\DEAKIN\TRIMESTER 1 2025\SIT378 TEAM PROJECT B\KatabaticGitRepo\sit378\lib\site-packages\xgboost\core.py:158: UserWarning: [13:18:23] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
2025-04-27 13:18:28,746 - INFO - Encoded categorical targets with mapping: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25

In [10]:
# 8. Extract and display TSTR results specifically
print("\n# 8. TSTR Performance Results")
tstr_results = get_tstr_results(evaluation_results)
if tstr_results is not None:
    print(tstr_results)


# 8. TSTR Performance Results
                    Accuracy        F1
LogisticRegression   0.02120  0.021474
RandomForest         0.03855  0.036997
MLP                  0.03580  0.033996
XGBoost              0.04285  0.039795


In [11]:
# 9. Save the synthetic data
print("\n# 9. Save synthetic data")
output_path = "letter_synthetic.csv"
synthetic_data.to_csv(output_path, index=False)
print(f"Synthetic data saved to {output_path}")

print("\nTest completed successfully!")


# 9. Save synthetic data
Synthetic data saved to letter_synthetic.csv

Test completed successfully!
